In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import re
import collections
import functools


In [2]:
#file load

f = open('data/content_list.csv','r')
line_counter = 0
unsupport_title_counter = 0
title_arr = []
ctr_arr = []

ex_hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
def map_hangul(str):
    str = str.replace('_',' ')
    str = ex_hangul.sub('',str)
    str = str.replace('  ',' ')
    return str

while True:    
    line = f.readline()
    if not line: break
    line_counter += 1
    
    line = line.replace('""',' ')
    content_info = line.split('"')
    
    try:
        title = map_hangul(content_info[1])
        title = ' '.join(title.split(' ')[2:])
        ctr = content_info[2].split(',')[1]
        title_arr.append(title)
        ctr_arr.append([float(ctr)])
        if line_counter % 10000 == 0:
            print(line_counter, ":", title, ctr)
    except:
        unsupport_title_counter += 1
        
print('총 라인 수 : ', line_counter)
print('타이틀 수 : ', len(title_arr))
print('CTR 수 : ', len(ctr_arr))
print('에러 수 : ',unsupport_title_counter)
    
print(title_arr[:5])
print(ctr_arr[:5])

10000 :  0.0043
20000 : 기자회견장에서 화보 찍는 체조요정들 0.0071
30000 : 스토리카드 그 사람이 사랑하는 방법 0.0128
40000 : 최수종 갓스트라 스브스뉴스 스토리카드  0.0071
50000 : 커피도 체온을 떨어뜨린다  동영상  0.0073
총 라인 수 :  51850
타이틀 수 :  51826
CTR 수 :  51826
에러 수 :  24
['초강력 워터프루프', '셔츠 초간단 연출법', '센스 넘치는 여행 소품들', '부위별 셀프 운동법', '']
[[0.0211], [0.0144], [0.0091], [0.0209], [0.0072]]


In [3]:
#dictionary를 만들기위해 단어만 주욱 모은다
#앞의 두 단어 제거 
words = functools.reduce(lambda x,y: x+' '+y, title_arr).split(' ')
print(len(words))
print(words[:10])

281748
['초강력', '워터프루프', '셔츠', '초간단', '연출법', '센스', '넘치는', '여행', '소품들', '부위별']


In [4]:
vocabulary_size = 50000

count = [['UNK', -1]] #unknown
count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
print(len(count))
print(count[:20])

#dictionary 정의 & feeding
dictionary = dict()
for word, _ in count:
    dictionary[word] = len(dictionary) #여기서 word는 count를 돌린거라서 유니크함
print(len(dictionary))

data = list()
unk_count = 0

for word in words:
    if word in dictionary:
        index = dictionary[word]
    else:
        index = 0  # dictionary['UNK']
        unk_count += 1
    data.append(index)
count[0][1] = unk_count
    
#title_arr 를 index값으로 title_vector_arr로 변환
title_vector_arr = []
for title in title_arr:
    title_vector = []
    word_arr = title.split(' ')
    for word in word_arr:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
        title_vector.append(index)
    title_vector_arr.append(title_vector)
    
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))

# del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample Data', data[:10])
print('Reverse dict', [reverse_dictionary[word_index] for word_index in data[:10]])
print(len(title_vector_arr))

50000
[['UNK', -1], ('', 16738), ('스토리카드', 6442), ('오후', 6333), ('오전', 5251), ('저녁', 5074), ('동영상', 2090), ('남자', 1157), ('중앙일보', 1125), ('여자', 1022), ('한줄', 883), ('가지', 677), ('이유', 671), ('한국일보', 664), ('왜', 612), ('내', 581), ('전자신문', 572), ('수', 557), ('년', 555), ('동아일보', 548)]
50000
Most common words (+UNK) [['UNK', 17086], ('', 16738), ('스토리카드', 6442), ('오후', 6333), ('오전', 5251)]
Sample Data [6870, 13411, 1052, 513, 2118, 2990, 1723, 59, 17088, 15967]
Reverse dict ['초강력', '워터프루프', '셔츠', '초간단', '연출법', '센스', '넘치는', '여행', '소품들', '부위별']
51826


In [5]:
# max_length = 0
# for title_vector in title_vector_arr:
#     if max_length < len(title_vector):
#         max_length = len(title_vector)
# print(max_length)

max_length = 10
#vector 사이즈를 정의하고, 부족한 부분은 empty_index로 채워서 사이즈를 맞춘다
empty_index = -1
title_vector_size = max_length
index = 0
for title_vector in title_vector_arr:
    while(len(title_vector) < title_vector_size):
        title_vector.append(empty_index)
    title_vector_arr[index] = title_vector[0:max_length]
    index +=1
print('Title vector array sample', title_vector_arr[:10])


Title vector array sample [[6870, 13411, -1, -1, -1, -1, -1, -1, -1, -1], [1052, 513, 2118, -1, -1, -1, -1, -1, -1, -1], [2990, 1723, 59, 17088, -1, -1, -1, -1, -1, -1], [15967, 602, 711, -1, -1, -1, -1, -1, -1, -1], [1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [7637, 12659, 4, -1, -1, -1, -1, -1, -1, -1], [3194, 30085, 3, -1, -1, -1, -1, -1, -1, -1], [36096, 16196, 40582, 9323, 45744, 0, 10279, 5, -1, -1], [2815, 0, 18422, 11894, 947, 8132, 156, 1007, 5, -1], [4482, 43, 1635, 21707, 459, 477, 5, -1, -1, -1]]


In [7]:
data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

13411 워터프루프 -> 1052 셔츠
13411 워터프루프 -> 6870 초강력
1052 셔츠 -> 513 초간단
1052 셔츠 -> 13411 워터프루프
513 초간단 -> 1052 셔츠
513 초간단 -> 2118 연출법
2118 연출법 -> 2990 센스
2118 연출법 -> 513 초간단


In [ ]:
def xavier_init(n_inputs, n_outputs):
    init_range = tf.sqrt(6.0/(n_inputs+n_outputs))
    return tf.random_uniform_initializer(-init_range, init_range)

grade_size = len(grade)

X = tf.placeholder(tf.float32, shape=(51826, title_vector_size), name='title_vector_arr')
Y = tf.placeholder(tf.float32, shape=(51826, grade_size), name='ctr_grade_arr')

neuron_width = 200
W1 = tf.get_variable(shape=[title_vector_size, neuron_width], initializer=xavier_init(title_vector_size, neuron_width), name='hidden1')
W2 = tf.get_variable(shape=[neuron_width, neuron_width], initializer=xavier_init(neuron_width, neuron_width), name='hidden2')
W3 = tf.get_variable(shape=[neuron_width, neuron_width], initializer=xavier_init(neuron_width, neuron_width), name='hidden3')
W4 = tf.get_variable(shape=[neuron_width, grade_size], initializer=xavier_init(neuron_width, grade_size), name='hidden4')

b1 = tf.Variable(tf.zeros([neuron_width]), name='bias1')
b2 = tf.Variable(tf.zeros([neuron_width]), name='bias2')
b3 = tf.Variable(tf.zeros([neuron_width]), name='bias3')
b4 = tf.Variable(tf.zeros([grade_size]), name='bias4')


In [19]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)  #train_inputs에 있는 id로 embeddings를 lookup 한 값을 가지고 있음
    print("embed : ",embed)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


embed :  Tensor("embedding_lookup:0", shape=(128, 128), dtype=float32, device=/device:CPU:0)


In [10]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  273.780822754
Nearest to 여성: 남녀싸움, 전기차들, 일부러, 소생시킬, 좋아하시나요, 더도, 고치려고, 세진다,
Nearest to 저녁: 패배원인, 쫀쫀한, 충동범죄, 칸으로, 합격에, 노른자를, 황재균유희관이, 애도,
Nearest to 법: 날씬했다, 그날의, 김종국에게, 멘탈에, 무장한, 기겁, 껌의, 황제의,
Nearest to 여행: 운동해야, 털리더라, 갈색, 곳은, 김연지, 동료와, 신입생들에, 후속,
Nearest to 스타투데이: 디오니소스, 끈끈했지만, 손마사지, 갈때, 대사하지마, 불타는, 보약, 여섯번째,
Nearest to 쉐어하우스: 바르니, 끝없다, 반토막, 파우더, 인하도, 체면에, 대하나, 전당된,
Nearest to 한국경제: 어민, 다리를, 맹수들앞에서, 아내와, 건조한, 집값, 울리다, 미궁,
Nearest to 중: 끝판대장, 순간포착, 시리즈, 말한, 김원준, 아이큐는, 청바지에, 인정받으려,
Nearest to 때: 물렀거라, 반응은, 추천해요, 도심, 마성의, 벗긴, 운용, 이하늬,
Nearest to 무슨: 대중이, 허리통증, 유망주, 닮아도, 슈퍼마리오, 애완견의, 여성호르몬의, 익혀,
Nearest to 다이어트: 베프반짝반짝, 상경, 회사를, 스크린에서, 예상, 잠을, 갇혔을, 사죄한,
Nearest to 첫: 걸어봤니, 환영교사는, 여자아이의, 하이파이브, 초로, 연예기자또, 약속이다, 베란다까지,
Nearest to 뉴스에이드: 낮이, 욕했어, 이순신, 소변기, 향수는, 최강의, 앞입니다, 노는,
Nearest to 만원: 마스크맨들, 확률은, 스캔들의, 잡으려, 고기도, 돌파구, 서러운데씁쓸한, 증상도,
Nearest to 하는: 파파라치포착, 싶고, 싶었어, 암은, 당신이거절당하는이유, 나이드는법, 현희씨의, 슬픈,
Nearest to 전: 일곱, 예능꾼들, 비슷한, 몸싸움해, 낳은, 아침부터, 위생, 표도르,


In [24]:
print(len(sim[0]))

50000


In [11]:
# Step 6: Visualize the embeddings.


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")